# Capítulo 6 - Ejemplos

In [ ]:
import warnings 
warnings.simplefilter('ignore') 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib
import seaborn as sns

## Leer los datos
Previamente, descargamos la base de datos con los casos de COVID19 desagregados por país y fecha de [este repositorio de Github](https://github.com/owid/covid-19-data/tree/master/public/data) y los guardamos en una carpeta llamada "data" contenida en el directorio donde estamos trabajando. Para leerlos utilizamos la función de `pandas` **read_csv**, a la cual se le pasa como parámetro indispensable el directorio donde están almacenados los datos seguido del nombre de la base de datos. A continuación podemos ver la tabla utilizando la función **head**. 

In [ ]:
df = pd.read_csv('./data/Covid_Data.csv') 
df.head()

In [ ]:
df.columns # Lista de columnas del dataframe

In [ ]:
df['location'].unique() # Despliega la lista de países en la base de datos

## Filtrado

Nos quedaremos sólo con cinco países, para esto filtraremos la base original. Usaremos la función isin, la cual regresa una serie booleana que muestra si el elemento correspondiente de la serie original es igual a algún elemento de la lista que toma como parámetro. Luego tomamos unicamente el subconjunto de índices correspondientes a los valores **True** de la serie. 

In [ ]:
country_list = ['Bolivia','Dominican Republic','El Salvador','Guatemala','Peru']
df = df[df['location'].isin(country_list)] 

## Fechas

In [ ]:
df['date'] # La columna de las fechas

Como vemos la columna fecha contiene datos tipo "objeto", sin embargo, `pandas` tiene funciones muy útiles para este tipo de datos. Convirtamos esta columna en datos tipo "datetime".

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['date']

Una vez que la columna tiene este formato, podemos conocer el día, mes, año y si está disponible, la hora, minutos etc. En este caso tenemos la fecha. También es posible conocer la semana del año en la que está el día determinado. 

In [ ]:
df['date'].dt.year

In [ ]:
df['date'].dt.isocalendar().week

In [ ]:
NewCasesCovid = df[['location','date','new_cases']]
NewCasesCovid

## Número de casos 
En la siguiente celda, graficaremos el número de casos nuevos por país, y la [media móvil](https://es.wikipedia.org/wiki/Media_m%C3%B3vil) semanal, la cual es una lista de números en la cual cada uno es el promedio de un subconjunto de los datos originales, en este caso por semana.

In [ ]:
for country in country_list:
    country_cases = NewCasesCovid[NewCasesCovid['location']==country]
    matplotlib.rc('xtick', labelsize=18)
    matplotlib.rc('ytick', labelsize=18)
    fig, ax = plt.subplots(figsize=(20,10))
    country_cases['Promedio móvil'] = country_cases['new_cases'].rolling(window=7).mean()
    ax.bar(country_cases['date'],country_cases['new_cases'],label = 'Número de casos Covid-19 '+country,color = '#BBD1DC')
    ax.plot(country_cases['date'],country_cases['Promedio móvil'],label = 'Media móvil 7 dias',color = '#EB610A')
    ax.set_title('Casos Covid-19 '+country,fontsize=16)
    #     ax.set_xticks(tickPts)
#     ax.set_xticklabels(tickLbls)

## Indice de restricciones
El índice de restricciones es una medida compuesta basada en nueve indicadores de respuesta que incluyen el cierre de escuelas, el cierre de lugares de trabajo y la prohibición de viajar, reescalados a un
valor de 0 a 100 (100 = más estricto). Para más información ver este [link](https://ourworldindata.org/grapher/covid-stringency-index). Acá podemos ver una comparación del número de casos diarios con el índice de restricciones y su cambio en el tiempo. 

In [ ]:
for country in country_list:
    country_cases = df[df['location']==country]
    fig, ax = plt.subplots(2,1,figsize=(20,10))
    country_cases['Promedio móvil'] = country_cases['new_cases'].rolling(window=7).mean()
    ax[0].bar(country_cases['date'],country_cases['new_cases'],label = 'Número de casos Covid-19 '+country,color = '#BBD1DC')
    ax[1].plot(country_cases['date'],country_cases['stringency_index'],label = 'Media móvil 7 dias',color = '#EB610A')
    ax[0
      ].plot(country_cases['date'],country_cases['Promedio móvil'],label = 'Media móvil 7 dias',color = '#EB610A')
    ax[0].set_title('Casos Covid-19 '+country,fontsize=16)
    ax[1].set_title('Indice de restricciones Covid-19 '+country,fontsize=16)

## Número de muertes - vacunas
Vemos en las gráficas que parece existir una cierta relación entre el número de casos nuevos y el índice de restricciones, ahondaremos más en esto, pero antes veamos la relación entre el número de muertes y la cantidad de personas vacunadas, la cual también esperamos que sea significativa.

In [ ]:
for country in country_list:
    country_cases = df[df['location']==country]
    fig, ax = plt.subplots(2,1,figsize=(20,10))
    country_cases['Promedio móvil'] = country_cases['new_deaths'].rolling(window=7).mean()
    ax[0].bar(country_cases['date'],country_cases['new_deaths'],label = 'Número de casos Covid-19 '+country,color = '#BBD1DC')
    ax[1].plot(country_cases['date'],country_cases['people_fully_vaccinated'],label = 'Media móvil 7 dias',color = '#EB610A')
    ax[0].plot(country_cases['date'],country_cases['Promedio móvil'],label = 'Media móvil 7 dias',color = '#EB610A')
    ax[0].set_title('Casos Covid-19 '+country,fontsize=16)
    ax[1].set_title('Cantidad de personas vacunadas Covid-19 '+country,fontsize=16)

## Algunas funciones estadísticas

Aunque `Python` tiene librerías especializadas en estadística, en `pandas` disponemos de algunas funciones sencillas. En el siguiente ejemplo hallamos la correlación entre las variables vistas anteriormente, ya que en las gráficas puede observarse cierta relación entre el número nuevo de casos y el índice de restricciones, y entre el número de muertes y la cantidad de personas vacunadas. Veamos si esta relación está respaldada estadísticamente. En los mapas de calor que siguen se puede ver en cada cuadro la correlación entre las variables que corresponden, podemos ver en cada uno que las correlaciones entre los nuevos casos y el índice de restricciones es negativa, y alta en varios países, lo cual puede significar que al disminuir las restricciones aumentan los casos positivos, y viceversa, al aumentar las restricciones disminuyen los casos positivos. También vemos que hay una correlación negativa entre la cantidad de muertes causadas por COVID y la cantidad de personas vacunadas. 
Otro aspecto a notar es la disminución, en algunos casos muy marcada del índice de restricciones a medida que aumenta el número de personas vacunadas. Lo cual puede explicar que el número de casos no disminuya en varios países. 

In [ ]:
for country in country_list:
    country_cases = df[df['location']==country]
    data = country_cases[['new_cases','new_deaths','stringency_index','people_fully_vaccinated']]
    fig,ax = plt.subplots()
    cor = data.corr(method='spearman') # La función corr halla la correlación entre las columnas del dataframe,
                                       # el método de correlación de spearman es escogido porque una correlación
                                       # de Pearson (lineal) no es adecuada en este caso 
    sns.heatmap(cor,annot=True,annot_kws={"fontsize":13})  # La librería seaborn proporciona una función muy 
                                                            # conveniente para graficar la matriz de correlaciones.
    ax.set_title(country)


## Diabetes
Una de las comorbilidades que empeoran la enfermedad de COVID19 es la diabetes, en nuestra base de datos disponemos de el índice medio de diabetes para cada país. Veamos la cantidad de muertes en relación con este dato, como la cantidad de muertes está en relación directa con la cantidad de casos nuevos, y esto puede variar por muchas razones, primero hallaremos el porcentaje de muertes con respecto a la cantidad total de casos. 
En la siguiente celda tomaremos el índice de prevalencia de diabetes por país, eliminando todos los que se repiten, ya que esta columna contiene un sólo número repetido todas las veces que el país aparece. 

In [ ]:
diabetes_prevalence_df = df[['location','diabetes_prevalence']].drop_duplicates(keep='first') # El parámetro keep es para decidir con cuál caso nos quedaremos

In [ ]:
df['new_deaths']/df['new_cases'] # Hallamos el radio muertes/casos

Como vemos tenemos varios "NaN", investiguemos más sobre esto, viendo cuántos hay, y en qué fechas están.

In [ ]:
df['new_deaths'].isnull().value_counts()

In [ ]:
null_values = df[df['new_deaths'].isnull()]
null_values['date'].value_counts()

La mayoría de los valores "Nan" están en las fechas de principios de la pandemia, en las cuales en muchos países aún no había casos, simplemente eliminaremos estos valores entonces.

In [ ]:
df.dropna(subset=['new_cases','new_deaths'],inplace=True) # La función dropna elimina todas las filas que contienen
                                                          # Nan en cualquiera de sus columnas, pero sólo queremos 
                                                          # eliminar los Nan de las columnas 'new_cases' y'new_deaths'
                                                          # para eso igualamos el parámetro subset a una lista de estas columnas.
df.reset_index(drop=True,inplace=True)

Probemos nuevamente a hallar el radio, ¡de nuevo tenemos valores nulos! 

In [ ]:
df['new_deaths']/df['new_cases']

In [ ]:
df[(df['new_deaths']/df['new_cases']).isnull()]['date'].value_counts()

In [ ]:
df[df['date']=='2022-01-06']

Cuando vemos en el dataframe las fechas en las que están estos valores nulos, vemos que el número de casos en esos días es igual a cero, esto puede deberse a que no se hicieron pruebas o no se reportó el número de casos nuevos por ser días festivos, entre otras muchas razones. Claramente, al dividir entre cero, obtenemos estos valores "Nan". Eliminemos entonces estos valores.

In [ ]:
df.drop(df[df['new_cases']==0].index,axis=0,inplace=True)

Ahora estamos en condiciones de hallar el radio buscado, primero agregaremos una columna a nuestro dataframe con el porcentaje de muertes de acuerdo al número de casos. Luego hacemos un diccionario que a cada país le asigna la media de este radio en el tiempo. Así, obtenemos un índice de cantidad de muertes con relación al número de casos positivos por país. Seguidamente convertimos este diccionario en una tabla, y con la función **merge** unimos la tabla de diabetes.  

In [ ]:
df['death_ratio'] = df['new_deaths']/df['new_cases']*100
dic = {country:df[df['location']==country]['death_ratio'].mean() for country in df['location'].unique()}
dic

In [ ]:
death_ratio_df = pd.DataFrame.from_dict(dic, orient='index').reset_index().rename(columns={'index':'location',0:'death_ratio'})

In [ ]:
death_ratio_df.merge(diabetes_prevalence_df,on='location')

Vemos que al menos para estos países no se observa una clara relación entre la prevalencia de diabetes y el índice de casos de muertes, esto nos dice, que aunque sin duda esta condición es una comorbilidad importante, y un riesgo para la salud en general, hay muchos otros factores a tener en cuenta cuando se quiere estudiar este tipo de fenómenos.  

## Recursos


- [Datos tipo fecha](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)
- [Media móvil](https://www.kite.com/python/answers/how-to-get-a-rolling-mean-from-a-pandas-dataframe-in--python)
- [Correlación](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)
- [Mapa de calor](https://seaborn.pydata.org/generated/seaborn.heatmap.html)